# notebook minist and different optimizers

the basis is kaiminig optimization and batchnorm

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from lib.modelmanager import *
from lib.model import *
from lib.data import *
from lib.optimizers import *
from lib.callbacks import *
from pathlib import Path

## mnist data

In [3]:
#x_train,y_train,x_valid,y_valid = load_pickled_train_valid_data(Path("/Users/kasparlund/.fastai/data/mnist.pkl.gz"))
x_train,y_train,x_valid,y_valid = load_pickled_train_valid_data(Path("C:/Users/kl/.fastai/data/mnist.pkl.gz"))
train_mean, train_sd = x_train.mean(), x_train.std()
x_train   = normalize( x_train, train_mean, train_sd)
x_valid   = normalize( x_valid, train_mean, train_sd)

x_train.mean(),x_train.std()

(tensor(-6.2598e-06), tensor(1.))

In [4]:
nh,bs              = 50,512
train_ds,valid_ds  = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
train_dl, valid_dl = ( DataLoader( train_ds, batch_size=bs, shuffle=True), 
                        DataLoader(valid_ds,  batch_size=bs*2) )
data               = DataBunch( train_dl, valid_dl, c_in=1, c_out=y_train.max().item()+1 )
print(len(train_dl)), print(len(valid_dl))

98
10


(None, None)

In [5]:
# average loss pr input sample at iteration 0
nb_classes = 10
initial_loss = -np.log( 1.0/nb_classes ) 
print(f"For equal distribution of cases pr class:\ninitial loss:    {initial_loss}\ninitial accuracy {1.0/nb_classes}")

For equal distribution of cases pr class:
initial loss:    2.3025850929940455
initial accuracy 0.1


In [6]:
mnist_view   = view_tfm(1,28,28)
layers_sizes = [8,16,32,32]
loss_func    = F.cross_entropy
sched        = combine_scheds([0.3, 0.7], [sched_cos(0.3, 0.6), sched_cos(0.6, 0.2)]) 

cbfs         = [TrainableModelCallback, TrainEvalCallback, OptimizerCallback, 
                partial(CudaCallback, device= torch.device('cuda',0)),
#                SimpleCudaCallback,
#                partial(ParamScheduler, 'lr', sched),
                partial(BatchTransformXCallback, tfm = mnist_view), 
#                partial(MixUp,α=0.4),
#                LR_Finder,
                Recorder, 
                partial(AvgStatsCallback,[accuracy]),
                ProgressCallback
               ]



## resnets + steppers=[sgd_step]

In [ ]:
opt = SGD(sched,max_lr=0.5)#0.5)
#loss_func=LabelSmoothingCrossEntropy()
loss_func=F.cross_entropy

layer = partial( conv_layer, stride=2, bn=False, zero_bn=False, act=GeneralRelu )
from torch import *
layer = partial( conv_layer, stride=2, bn=False, zero_bn=False, act=partial(torch.nn.ReLU,inplace=True) )
model = partial(xresnet18, c_in=data.c_in, c_out=data.c_out)()
#model = get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer)
mm    = CnnModelManager( model )
mm.initialize(is_resnet=True)


xb,_ = mm.getFirstbatch( data, normalization = mnist_view)
print(f"xb.shape:{xb.shape}")
mm.summary(xb, print_mod=False)
learn = Learner( mm.model, data, loss_func=loss_func)

%time learn.fit(25, opt=opt, cb_funcs=cbfs)

In [ ]:
#learn.find_subcription_by_cls(LR_Finder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()

## steppers=[sgd_step]

In [ ]:
mm    = CnnModelManager( get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer) )
mm.initialize(is_resnet=False)
opt = SGD(sched,max_lr=0.8)#0.5)

#loss_func=LabelSmoothingCrossEntropy()
loss_func=F.cross_entropy
learn = Learner( mm.model, data, loss_func=loss_func)

%time learn.fit(3, opt=opt,cb_funcs=cbfs)

In [ ]:
#learn.find_subcription_by_cls(LR_Finder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()

## steppers=[weight_decay, sgd_step]

In [ ]:
mm    = CnnModelManager( get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer) )
mm.initialize(is_resnet=False)

opt = SGD(sched,max_lr=0.5, max_wd=0.01)
#print(opt.hypers[0]['lr']), print(opt.hypers[0]['wd'])
learn = Learner( mm.model, data, loss_func=F.cross_entropy)
%time learn.fit(1, opt=opt,cb_funcs=cbfs)

In [ ]:
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()

## momentum: steppers=[momentum_step,weight_decay]

In [ ]:
mm    = CnnModelManager( get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer) )
mm.initialize(is_resnet=False)

opt   = SGD_Momentum(sched,max_lr=1.0, moms=(0.85,0.95), max_wd=1e-3 )
learn = Learner( mm.model, data, loss_func=F.cross_entropy)
%time learn.fit(1, opt=opt, cb_funcs=cbfs)

In [ ]:
#learn.find_subcription_by_cls(LR_Finder).plot_loss(skip_end=0),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()

## Adam: steppers=[adam_step]

In [ ]:
activ_func = partial(ReLUOffset) 
#activ_func = partial(nn.ReLU,inplace=True)
model = partial(xresnet18, c_in=data.c_in, c_out=data.c_out, activ_func=activ_func)()
#model = get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer)
mm    = CnnModelManager( model )
mm.initialize(is_resnet=True)

opt   = Adam(sched,max_lr=3e-3, moms=(0.85,0.95), max_wd = 0)#1e-6)
learn = Learner( mm.model, data, loss_func=F.cross_entropy)
#%time learn.fit(10, opt=opt, cb_funcs=cbfs)
%time learn.fit(100, opt=opt, cb_funcs=cbfs.copy()+[partial(MixUp,α=0.01)])

#%time learn.fit(1, opt=opt, cb_funcs=cbfs.copy()+[LRFinder])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.132474,0.709220,0.306711,0.960500,00:03
1,0.227979,0.967740,0.118000,0.977600,00:03
2,0.139760,0.978360,0.086372,0.981400,00:03
3,0.113617,0.983380,0.068686,0.982300,00:03
4,0.101348,0.986720,0.059495,0.984400,00:03
5,0.093826,0.988200,0.058919,0.983900,00:03
6,0.091188,0.989340,0.055250,0.985500,00:03
7,0.084468,0.991000,0.048234,0.987700,00:03
8,0.080122,0.992440,0.046802,0.986600,00:03
9,0.076858,0.992580,0.054112,0.985500,00:03


In [ ]:
#learn.find_subcription_by_cls(LRFinder).plot_loss(skip_end=10),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()
#learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()

## LAMB: steppers=[lamb_step]

It's then super easy to implement a new optimizer. This is LAMB from a [very recent paper](https://arxiv.org/pdf/1904.00962.pdf):

$\begin{align}
g_{t}^{l} &= \nabla L(w_{t-1}^{l}, x_{t}) \\
m_{t}^{l} &= \beta_{1} m_{t-1}^{l} + (1-\beta_{1}) g_{t}^{l} \\
v_{t}^{l} &= \beta_{2} v_{t-1}^{l} + (1-\beta_{2}) g_{t}^{l} \odot g_{t}^{l} \\
m_{t}^{l} &= m_{t}^{l} / (1 - \beta_{1}^{t}) \\
v_{t}^{l} &= v_{t}^{l} / (1 - \beta_{2}^{t}) \\
r_{1} &= \|w_{t-1}^{l}\|_{2} \\
s_{t}^{l} &= \frac{m_{t}^{l}}{\sqrt{v_{t}^{l} + \epsilon}} + \lambda w_{t-1}^{l} \\ 
r_{2} &= \| s_{t}^{l} \|_{2} \\
\eta^{l} &= \eta * r_{1}/r_{2} \\ 
w_{t}^{l} &= w_{t}^{l-1} - \eta_{l} * s_{t}^{l} \\
\end{align}$

In [ ]:
mm    = CnnModelManager( get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer) )
mm.initialize(is_resnet=False)

opt   = LAMB(sched,max_lr=0.02, moms=(0.85,0.95), max_wd = 1e-4)     
#opt   = LAMB(sched,max_lr=0.005, moms=(0.85,0.95), max_wd = 1e-4)     #with mixup
learn = Learner( mm.model, data, loss_func=F.cross_entropy)
%time learn.fit(1, opt=opt, cb_funcs=cbfs)

In [ ]:
learn.find_subcription_by_cls(Recorder).plot_loss(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()

## mnist with adam and hooks

In [ ]:
#activ_func = partial(nn.ReLU,inplace=True) 
activ_func = partial(ReLUOffset) 
model = partial(xresnet18, c_in=data.c_in, c_out=data.c_out, activ_func=activ_func)()
#model = get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer)
mm    = CnnModelManager( model )
mm.initialize(is_resnet=True)# mm.initialize(is_resnet=False)

#model = partial(xresnet18, c_in=data.c_in, c_out=data.c_out)()
#opt   = Adam(sched,max_lr=0.05, moms=(0.85,0.95), max_wd = 1e-3)
opt   = Adam(sched,max_lr=3e-3, moms=(0.85,0.95), max_wd = 0) #1e-5) # xresnet18
learn = Learner( mm.model, data, loss_func=F.cross_entropy)
with Hooks(mm.model, append_stats) as hooks: 
    #learn.fit(3, opt=opt, cb_funcs=cbfs.copy()+[partial(MixUp,α=0.01)])    
    learn.fit(3, opt=opt, cb_funcs=cbfs)    
    plot_layer_stats( hooks )

## mnist with LAMB and hooks

In [ ]:
#model = CnnModelManager( partial(xresnet18, c_in=data.c_in, c_out=data.c_out)() )
mm    = CnnModelManager( get_cnn_model(layers_sizes, data.c_in, data.c_out, layer=layer) )
mm.initialize(is_resnet=False)

opt   = LAMB(sched,max_lr=0.008, moms=(0.85,0.95), max_wd = 1e-6)     
learn = Learner( mm.model, data, loss_func=F.cross_entropy)
with Hooks(mm.model, append_stats) as hooks: 
    learn.fit(3, opt=opt, cb_funcs=cbfs)
    plot_layer_stats( hooks )